In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
data_cf = {
    'User': ['Alice', 'Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie'],
    'Item': ['Movie1', 'Movie2', 'Movie3', 'Movie2', 'Movie3', 'Movie1', 'Movie4'],
    'Rating': [5, 3, 4, 4, 5, 5, 4]
}
df_cf = pd.DataFrame(data_cf)
user_item_matrix = df_cf.pivot(index='User', columns='Item', values='Rating').fillna(0)
user_similarity = user_item_matrix.dot(user_item_matrix.T)
norms = np.array([np.sqrt(np.diagonal(user_similarity))])
user_similarity = user_similarity / norms.T / norms
def get_recommendations_cf(user, user_item_matrix, user_similarity):
    if user not in user_item_matrix.index:
        return f"User '{user}' not found in the dataset."
    similar_users = user_similarity.loc[user].sort_values(ascending=False).index[1:]
    recommendations = pd.Series(dtype=float)
    for similar_user in similar_users:
        recommendations = pd.concat([recommendations, user_item_matrix.loc[similar_user][user_item_matrix.loc[similar_user] > 0]])
    recommendations = recommendations.groupby(recommendations.index).mean().sort_values(ascending=False)
    return recommendations.head(3)
books = pd.DataFrame({
    'Title': ['Book1', 'Book2', 'Book3', 'Book4'],
    'Description': [
        'A story about magic and adventure',
        'A deep dive into technology and innovation',
        'An exploration of human psychology',
        'A tale of love and relationships'
    ]
})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['Description'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
def get_content_based_recommendations(title, books, cosine_sim):
    if title not in books['Title'].values:
        return f"Book '{title}' not found in the dataset."
    idx = books.index[books['Title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]
    book_indices = [i[0] for i in sim_scores]
    return books['Title'].iloc[book_indices]
if __name__ == "__main__":
    user = 'Alice'
    print(f"Collaborative Filtering Recommendations for {user}:")
    print(get_recommendations_cf(user, user_item_matrix, user_similarity))
    book_title = 'Book1'
    print(f"\nContent-Based Filtering Recommendations for '{book_title}':")
    print(get_content_based_recommendations(book_title, books, cosine_sim))

Collaborative Filtering Recommendations for Alice:
Movie1    5.0
Movie3    5.0
Movie2    4.0
dtype: float64

Content-Based Filtering Recommendations for 'Book1':
1    Book2
2    Book3
3    Book4
Name: Title, dtype: object
